This is a notebook version of a lab/stencil file from the textbook "Coding the Matrix"

These files should NOT be made public, both to avoid sharing solutions, and to allow the book to maintain its value.
 
This stencil for the lab was originally created by Philip Klien, who holds the copyright.

This is a derivative work (converting to a notebook), designed for classroom simplicity, not as a separate product.

# Lab: Comparing Voting Records Using Dot-Products

### Importing the necessary data

In [214]:
import numpy as np
# Requirements: the data file 'voting_record_dump109.txt' is included in the same folder as the notebook
f = open('../Datasets/US_Senate_voting_data_109.txt')
mylist = list(f)

### Task 1: Create Voting Dictionary

In [163]:
def parse_senator(senator, party_designation=None, verbose=False):    
    s_array = senator.split()
    
    #ints of the character votes
    int_votes = list(map(int,s_array[3:]))
    
    #keep name and voting history
    if verbose:
        s_key = (s_array[0], s_array[2])
    else:
        s_key = s_array[0]
    s_voting_dict_elmt = (s_key, int_votes)
    if party_designation:
        if s_array[1] == party_designation:
            return s_voting_dict_elmt
        #not a party we care about
        else:
            return (None,None)
    else:
        return s_voting_dict_elmt
        
        
def create_voting_dict(strlist, party_flag=None):
    """
    Input: a list of strings.  Each string represents the voting record of a senator.
           The string consists of 
              - the senator's last name, 
              - a letter indicating the senator's party,
              - a couple of letters indicating the senator's home state, and
              - a sequence of numbers (0's, 1's, and negative 1's) indicating the senator's
                votes on bills
              all separated by spaces.
              
    Output: A dictionary that maps the last name of a senator
            to a list of numbers representing the senator's voting record.
    Example: 
        >>> vd = create_voting_dict(['Kennedy D MA -1 -1 1 1', 'Snowe R ME 1 1 1 1'])
        >>> vd == {'Snowe': [1, 1, 1, 1], 'Kennedy': [-1, -1, 1, 1]}
        True

    You can use the .split() method to split each string in the
    strlist into a list; the first element of the list will be the senator's
    name, the second will be his/her party affiliation (R or D), the
    third will be his/her home state, and the remaining elements of
    the list will be that senator's voting record on a collection of bills.

    You can use the built-in procedure int() to convert a string
    representation of an integer (e.g. '1') to the actual integer
    (e.g. 1).

    The lists for each senator should preserve the order listed in voting data.
    In case you're feeling clever, this can be done in one line.
    """
    voting_dict = dict(map(lambda sen_str : parse_senator(*[sen_str, party_flag]), strlist))
    
    voting_dict.pop(None, None)
    
    return voting_dict
    
    

### Test Code for Task #1
The following code can be used to test your 'create_voting_dict' procedure. It should be removed before exporting to the script file for auto-grading.

In [164]:
vd = create_voting_dict(['Kennedy D MA -1 -1 1 1', 'Snowe R ME 1 1 1 1'])
print(vd)
vd == {'Snowe': [1, 1, 1, 1], 'Kennedy': [-1, -1, 1, 1]}

{'Snowe': [1, 1, 1, 1], 'Kennedy': [-1, -1, 1, 1]}


True

### Task 2: Write a policy compare procedure

In [129]:
import numpy as np

def policy_compare(sen_a, sen_b, voting_dict):
    """
    Input: last names of sen_a and sen_b, and a voting dictionary mapping senator
           names to lists representing their voting records.
    Output: the dot-product (as a number) representing the degree of similarity
            between two senators' voting policies
    Example:
        >>> voting_dict = {'Fox-Epstein':[-1,-1,-1,1],'Ravella':[1,1,1,1]}
        >>> policy_compare('Fox-Epstein','Ravella', voting_dict)
        -2
    
    The code should correctly compute the dot-product even if the numbers are not all in {0,1,-1}.
        >>> policy_compare('A', 'B', {'A':[100,10,1], 'B':[2,5,3]})
        253
        
    You should definitely try to write this in one line.
    """
    return np.dot(voting_dict[sen_a], voting_dict[sen_b])
    

### Test code for Task #2
The following code can be used to test 'policy_compare'

In [130]:
voting_dict = {'Fox-Epstein':[-1,-1,-1,1],'Ravella':[1,1,1,1]}
print(policy_compare('Fox-Epstein','Ravella', voting_dict))
print(policy_compare('A', 'B', {'A':[100,10,1], 'B':[2,5,3]}))
print(policy_compare('A', 'B', {'A':[1,1,1], 'B':[1,1,-1]}))

-2
253
1


### Task 3: Most Similar Procedure

In [179]:
def most_similar(sen, voting_dict):
    """
    Input: the last name of a senator, and a dictionary mapping senator names
           to lists representing their voting records.
    Output: the last name of the senator whose political mindset is most
            like the input senator (excluding, of course, the input senator
            him/herself). Resolve ties arbitrarily.
    Example:
        >>> vd = {'Klein': [1,1,1], 'Fox-Epstein': [1,-1,0], 'Ravella': [-1,0,0]}
        >>> most_similar('Klein', vd)
        'Fox-Epstein'
        >>> vd == {'Klein': [1,1,1], 'Fox-Epstein': [1,-1,0], 'Ravella': [-1,0,0]}
        True
        >>> vd = {'a': [1,1,1,0], 'b': [1,-1,0,0], 'c': [-1,0,0,0], 'd': [-1,0,0,1], 'e': [1, 0, 0,0]}
        >>> most_similar('c', vd)
        'd'

    Note that you can (and are encouraged to) re-use your policy_compare procedure.
    """
    # copy dict but remove exisint Senator that we are comparing
    # helpful for more consice iteration below
    others = voting_dict.copy()
    others.pop(sen)
    sen_names = others.keys()

    # start with the first Senator in the dict without our given Senator
    # maintain intermmediate twin with most comparable voting history
    twin_name = others.popitem()[0]
    twin_score = policy_compare(sen, twin_name, voting_dict)
        
    # iterate through senators and update the twin upon discovery of a more matched voter.
    for name in sen_names:
        voting_delta = policy_compare(sen, name, voting_dict)
        if (voting_delta > twin_score):
            twin_name = name
            twin_score = voting_delta
        
    return twin_name

Testing Code

In [180]:
vd = {'Klein': [1,1,1], 'Fox-Epstein': [1,-1,0], 'Ravella': [-1,0,0]}
print(most_similar('Klein', vd))  #Expect output to be 'Fox-Epstein'
print(vd == {'Klein': [1,1,1], 'Fox-Epstein': [1,-1,0], 'Ravella': [-1,0,0]} )  #Expected output, 'True'

vd = {'a': [1,1,1,0], 'b': [1,-1,0,0], 'c': [-1,0,0,0], 'd': [-1,0,0,1], 'e': [1, 0, 0,0]}
print(most_similar('c', vd)) #Expected output, 'd'

Fox-Epstein
True
d


### Task 4: Least Similar Procedure

In [85]:
def least_similar(sen, voting_dict, score_required=False):
    """
    Input: the last name of a senator, and a dictionary mapping senator names
           to lists representing their voting records.
    Output: the last name of the senator whose political mindset is least like the input
            senator.
    Example:
        >>> vd = {'a': [1,1,1], 'b': [1,-1,0], 'c': [-1,0,0]}
        >>> least_similar('a', vd)
        'c'
        >>> vd == {'a': [1,1,1], 'b': [1,-1,0], 'c': [-1,0,0]}
        True
        >>> vd = {'a': [-1,0,0], 'b': [1,0,0], 'c': [-1,1,0], 'd': [-1,1,1]}
        >>> least_similar('c', vd)
        'b'
    """
    # copy dict but remove exisint Senator that we are comparing
    # helpful for more consice iteration below
    others = voting_dict.copy()
    others.pop(sen)
    sen_names = others.keys()

    # start with the first Senator in the dict without our given Senator
    # maintain intermmediate twin with most comparable voting history
    twin_name = others.popitem()[0]
    twin_score = policy_compare(sen, twin_name, voting_dict)
        
    # iterate through senators and update the twin upon discovery of a more matched voter.
    for name in sen_names:
        voting_delta = policy_compare(sen, name, voting_dict)
        if (voting_delta < twin_score):
            twin_name = name
            twin_score = voting_delta
        
    return (twin_name, twin_score) if score_required else twin_name


In [86]:
vd = {'a': [1,1,1], 'b': [1,-1,0], 'c': [-1,0,0]}
print(least_similar('a', vd)) #expected output 'c'
print( vd == {'a': [1,1,1], 'b': [1,-1,0], 'c': [-1,0,0]} ) #Expected output 'True'

vd = {'a': [-1,0,0], 'b': [1,0,0], 'c': [-1,1,0], 'd': [-1,1,1]}
print(least_similar('c', vd)) #Expected output 'b'

c
True
b


### Task 5: Senators Chafee and Santorum

In [87]:
most_like_chafee = most_similar('Chafee', create_voting_dict(mylist))
print(most_like_chafee)

least_like_santorum = least_similar('Chafee', create_voting_dict(mylist))
print(least_like_santorum)

Jeffords
Sununu


### Task 6: Most Average Democrat

In [154]:
import numpy as np

def find_average_similarity(sen, sen_set, voting_dict):
    """
    Input: the name of a senator, a set of senator names, and a voting dictionary.
    Output: the average dot-product between sen and those in sen_set.
    Example:
        >>> vd = {'Klein':[1,1,1], 'Fox-Epstein':[1,-1,0], 'Ravella':[-1,0,0], 'Oyakawa':[-1,-1,-1], 'Loery':[0,1,1]}
        >>> sens = {'Fox-Epstein','Ravella','Oyakawa','Loery'}
        >>> find_average_similarity('Klein', sens, vd)
        -0.5
        >>> sens == {'Fox-Epstein','Ravella', 'Oyakawa', 'Loery'}
        True
        >>> vd == {'Klein':[1,1,1], 'Fox-Epstein':[1,-1,0], 'Ravella':[-1,0,0], 'Oyakawa':[-1,-1,-1], 'Loery':[0,1,1]}
        True
    """
    n = len(sen_set)
    v_sum = 0.0
    
    for s in sen_set:
        v_sum += np.dot(voting_dict[sen], voting_dict[s])
        
    return v_sum/n
        


In [155]:
#Test Code
vd = {'Klein':[1,1,1], 'Fox-Epstein':[1,-1,0], 'Ravella':[-1,0,0], 'Oyakawa':[-1,-1,-1], 'Loery':[0,1,1]}
sens = {'Fox-Epstein','Ravella','Oyakawa','Loery'}

print(find_average_similarity('Klein', sens, vd) ) 
# Expected output :   -0.5

print(sens == {'Fox-Epstein','Ravella', 'Oyakawa', 'Loery'} ) 
#Expected output:   True

print(vd == {'Klein':[1,1,1], 'Fox-Epstein':[1,-1,0], 'Ravella':[-1,0,0], 'Oyakawa':[-1,-1,-1], 'Loery':[0,1,1]} ) 
#expected output:   True\

-0.5
True
True


In [187]:
# give the last name (or code that computes the last name)
dem_dict = create_voting_dict(mylist, 'D')
dem_sen_names = dem_dict.keys();

name0 = dem_sen_names.pop()
score0 = find_average_similarity(name0, dem_sen_names, dem_dict)
most_avg_dem = (name0, score0)

for dem in dem_sen_names:
    score = find_average_similarity(dem, dem_sen_names, dem_dict)
    if score > most_avg_dem[1]:
        most_avg_dem = (dem, score)

print(most_avg_dem)

('Biden', 34.976190476190474)


### Task 7: Average Record

In [188]:
def find_average_record(sen_set, voting_dict):
    """
    Input: a set of last names, a voting dictionary
    Output: a vector containing the average components of the voting records
            of the senators in the input set
    Example: 
        >>> voting_dict = {'Klein': [-1,0,1], 'Fox-Epstein': [-1,-1,-1], 'Ravella': [0,0,1]}
        >>> senators = {'Fox-Epstein','Ravella'}
        >>> find_average_record(senators, voting_dict)
        [-0.5, -0.5, 0.0]
        >>> voting_dict == {'Klein': [-1,0,1], 'Fox-Epstein': [-1,-1,-1], 'Ravella': [0,0,1]}
        True
        >>> senators
        {'Fox-Epstein','Ravella'}
        >>> d = {'c': [-1,-1,0], 'b': [0,1,1], 'a': [0,1,1], 'e': [-1,-1,1], 'd': [-1,1,1]}
        >>> find_average_record({'a','c','e'}, d)
        [-0.6666666666666666, -0.3333333333333333, 0.6666666666666666]
        >>> find_average_record({'a','c','e','b'}, d)
        [-0.5, 0.0, 0.75]
        >>> find_average_record({'a'}, d)
        [0.0, 1.0, 1.0]
    """        
    
    voting_list_set = map(voting_dict.get,sen_set)
   
    n = len(sen_set) * 1.0    
    return np.sum(voting_list_set, axis=0) / n

In [189]:
#Test your procedure here...
#Use the previous tasks descriptions, and their test code to determine what you should extract from the above definition cell.
# d = {'c': [-1,-1,0], 'b': [0,1,1], 'a': [0,1,1], 'e': [-1,-1,1], 'd': [-1,1,1]}
# find_average_record({'a','c','e'}, d)

voting_dict = {'Klein': [-1,0,1], 'Fox-Epstein': [-1,-1,-1], 'Ravella': [0,0,1]}
senators = {'Fox-Epstein','Ravella'}
print(find_average_record(senators, voting_dict))

[-0.5 -0.5  0. ]


In [190]:
dem_dict = create_voting_dict(mylist, 'D')
avg_dem_rec = find_average_record(dem_dict.keys(), dem_dict)
print(avg_dem_rec)

[-0.1627907  -0.23255814  1.          0.8372093   0.97674419 -0.13953488
 -0.95348837  0.81395349  0.97674419  0.97674419  0.90697674  0.76744186
  0.6744186   0.97674419 -0.51162791  0.93023256  0.95348837  0.97674419
 -0.39534884  0.97674419  1.          1.          1.          0.95348837
 -0.48837209  1.         -0.3255814  -0.06976744  0.97674419  0.86046512
  0.97674419  0.97674419  1.          1.          0.97674419 -0.34883721
  0.97674419 -0.48837209  0.23255814  0.88372093  0.44186047  0.90697674
 -0.90697674  1.          0.90697674 -0.30232558]


### Task 8: Bitter Rivals

In [217]:
def bitter_rivals(voting_dict):
    """
    Input: a dictionary mapping senator names to lists representing
           their voting records
    Output: a tuple containing the two senators who most strongly
            disagree with one another.
    Example: 
        >>> voting_dict = {'Klein':[-1,0,1], 'Fox-Epstein':[-1,-1,-1], 'Ravella':[0,0,1], 'Oyakawa':[1,1,1], 'Loery':[1,1,0]}
        >>> br = bitter_rivals(voting_dict)
        >>> br == ('Fox-Epstein', 'Oyakawa') or br == ('Oyakawa', 'Fox-Epstein')
        True
    """
    
    # LinAlg implementation
    
    # data fetch
    rival_name_list = voting_dict.keys()
    voting_matrix = np.matrix(voting_dict.values())  
    
    # compute a matrix lookup for senator comparison ratings
    rival_matrix = np.dot(voting_matrix, np.transpose(voting_matrix))
        
    # find the index of the minimals comparison ratings
    # , which corresponds to the 2 most differing senators
    rival_indices = np.where(rival_matrix == rival_matrix.min())[0]
    rivals = rival_name_list[rival_indices[0]], rival_name_list[rival_indices[1]]

    return rivals
    

    # quick python implementation:
        
#     sen_rival_scores = [(sen, least_similar(sen, voting_dict, True)) for sen in voting_dict.keys()]        
#     bitter_rivals = min(sen_rival_scores, key=lambda tup: tup[1][1])
#     return (bitter_rivals[0], bitter_rivals[1][0])
    


In [218]:
#Again, test your 'bitter_rivals' procedure here!
voting_dict = {'Klein':[-1,0,1], 'Fox-Epstein':[-1,-1,-1], 'Ravella':[0,0,1], 'Oyakawa':[1,1,1], 'Loery':[1,1,0]}
br = bitter_rivals(voting_dict)
br == ('Fox-Epstein', 'Oyakawa') or br == ('Oyakawa', 'Fox-Epstein')

True

### Task 9: Open Ended-Study
Create a separate notebook file that answers the open-ended study questions posed in the lab handout.
You should turn in BOTH the code, and the explanations of your answers (i.e. more than just 'yes', 'no', and names!)

    | 1) Who is most R/D:
        a) Senator? b) State?
    | 2) Is McCains a Maverick?
    | 3) Is Obama Extremist?
    | 4) Who has most opponents (based on some threshold)?


### Turning in your work...
If you get to this point... awesome! We'll go over how to submit the code files for grading, and where to submit the explanations either Thursday or next Tuesday. 
This assignment is tentatively due by 10pm next Tuesday. 

In [375]:
def create_verbose_voting_dict(strlist, party_flag=None):
    voting_dict = dict(map(lambda sen_str : parse_senator(*[sen_str, party_flag, True]), strlist))
    voting_dict.pop(None, None)
    return voting_dict
        

repubs = create_verbose_voting_dict(mylist, 'R')
dems = create_verbose_voting_dict(mylist, 'D')

def avg_helper(party_dict, avg_rec, key=('party', 'USA')):
    dict_proxy = party_dict.copy()
    dict_proxy.update([(key, avg_rec)])
    most_avg_member = most_similar(key, dict_proxy)
    return most_avg_member

def party_avg_h(party_dict, key=('party', 'USA')):
    avg_rec = find_average_record(party_dict.keys(), party_dict)
    return avg_helper(party_dict, avg_rec, key)


print("AVG_DEM", party_avg_h(dems))
print("AVG_REP", party_avg_h(repubs))

voting_dict = dems.copy()
voting_dict.update(repubs.copy())
rivals = bitter_rivals(voting_dict)
print("RIVALS", rivals)

print("Rival1 Score", find_average_similarity(rivals[0], ordered_sen_names, voting_dict))
print("Rival2 Score", find_average_similarity(rivals[1], ordered_sen_names, voting_dict))

print("Obama Score", find_average_similarity(('Obama', 'IL'), ordered_sen_names, voting_dict))
print("McCain Score", find_average_similarity(('McCain', 'AZ'), ordered_sen_names, voting_dict))

ordered_sen_names = voting_dict.keys();
voting_matrix = voting_dict.values()
rival_matrix = np.dot(voting_matrix, np.transpose(voting_matrix))

states = dict()
for sen in ordered_sen_names:
    state_name = sen[1]
    votes = voting_dict[sen]
    if states.has_key(state_name):
        state = states[state_name]
        state.append(votes)
    else:
        states[state_name] = [votes]
        
states = {k: np.mean(v, axis=0) for k, v in states.items()}
avg_dem = find_average_record(dems.keys(), dems)
avg_rep = find_average_record(repubs.keys(), repubs)

print(avg_helper(states, avg_rec=avg_dem))
print(avg_helper(states, avg_rec=avg_rep))

print(np.dot(states['MD'], voting_dict[('Biden', 'DE')]))
print(np.dot(states['MO'], voting_dict[('Grassley', 'IA')]))


# rival_check_indices = np.where(rival_matrix == rival_matrix.min())[0]
# print ordered_sen_names[rival_check_indices[0]]
# print ordered_sen_names[rival_check_indices[1]]

def opponents(threshold, ordered_sen_names, rival_matrix):
    most_hated = (ordered_sen_names[0], 0)

    for name, sen in zip(ordered_sen_names, rival_matrix):
        opps = np.where(sen < threshold)
        n_opps = len(opps[0])
        if n_opps > most_hated[1]:
            most_hated = (name, n_opps)
    return most_hated

for i in range(10, 50, 5):
    print("PUBLIC ENEMY @ PERCENT", i, opponents(i, ordered_sen_names, rival_matrix))

('AVG_DEM', ('Biden', 'DE'))
('AVG_REP', ('Grassley', 'IA'))
('RIVALS', (('Feingold', 'WI'), ('Inhofe', 'OK')))
('Rival1 Score', 14.010204081632653)
('Rival2 Score', 22.459183673469386)
('Obama Score', 25.479591836734695)
('McCain Score', 23.020408163265305)
MD
MO
42.0
46.0
('PUBLIC ENEMY @ PERCENT', 10, (('Feingold', 'WI'), 51))
('PUBLIC ENEMY @ PERCENT', 15, (('Feingold', 'WI'), 57))
('PUBLIC ENEMY @ PERCENT', 20, (('Feingold', 'WI'), 62))
('PUBLIC ENEMY @ PERCENT', 25, (('Feingold', 'WI'), 72))
('PUBLIC ENEMY @ PERCENT', 30, (('Coburn', 'OK'), 94))
('PUBLIC ENEMY @ PERCENT', 35, (('Lieberman', 'CT'), 97))
('PUBLIC ENEMY @ PERCENT', 40, (('Lieberman', 'CT'), 98))
('PUBLIC ENEMY @ PERCENT', 45, (('McCain', 'AZ'), 98))
